In [85]:
import pandas as pd
import altair as alt
import datetime

In [86]:
df = pd.read_excel('2014-2020.xlsx')
df.head()

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Unnamed: 12
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0,NaN
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0,NaN
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0,NaN
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0,NaN
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0,NaN


In [87]:
df.drop(['GAES_GEN', 'GAES_PUMP', 'UK_BLR_RUS','UK_EURO', 'UK_MLD'], axis = 1, inplace = True)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.columns = ['Час/Дата', 'AES', 'TEC', 'VDE', 'TES', 'GES', 'CONSUMPTION']
df = df.rename(columns={'Час/Дата': 'Time/Date'})
df.head()

,Time/Date,AES,TEC,VDE,TES,GES,CONSUMPTION
0,24-31.12.2020,9235,2039,621,4942,385.0,16693
1,23-31.12.2020,9221,2159,707,5549,470.0,17805
2,22-31.12.2020,9249,2377,709,5906,1000.0,18870
3,21-31.12.2020,9256,2499,702,6329,909.0,19887
4,20-31.12.2020,9213,2521,702,6640,823.0,20387


In [88]:
df['Year'] = df['Time/Date'].apply(lambda x: str(x)[-4:])
df.head()

,Time/Date,AES,TEC,VDE,TES,GES,CONSUMPTION,Year
0,24-31.12.2020,9235,2039,621,4942,385.0,16693,2020
1,23-31.12.2020,9221,2159,707,5549,470.0,17805,2020
2,22-31.12.2020,9249,2377,709,5906,1000.0,18870,2020
3,21-31.12.2020,9256,2499,702,6329,909.0,19887,2020
4,20-31.12.2020,9213,2521,702,6640,823.0,20387,2020


# 1. Як змінювалась структура генерації електроенергії за роками?

Виконувавши це завдання, я розглядала кілька варіантів візуалізації. Спочатку найкращим варіантом мені видався grouped bar chart, проте 7 груп на графіку виглядали надто громіздко. Тоді я обирала між line graph та stacked bar chart та обрала другий варіант. На такому графіку можна легко порівняти категорії за кольором та загальну кількість електроенергії. Є проблема з тим, що, починаючи з другого виду електростанцій, відлік кількості електроенергії йде не з нуля, і можуть виникнути труднощі з порівнянням. Для цього я додала tooltip, якщо виникають запитання можна навести на колонку і дізнатися точку кількість електроенергії.

In [89]:
df1 = df.copy(deep=False)
df1.drop(['Time/Date', 'CONSUMPTION'], axis = 1, inplace = True)

df_sum = df1.groupby("Year").sum()
df_sum.head()

,AES,TEC,VDE,TES,GES
Year,,,,,
2014,88204418,14684731,1606386,68605877,7991074.0
2015,87413763,12041074,1234936,50259819,4964478.0
2016,80762094,12841033,1093190,49879543,7113989.0
2017,85314258,12208062,1177032,44945303,8531952.0
2018,84351328,12261018,1863726,47747092,10072767.0


In [90]:
df_sum = df_sum.reset_index()
df_sum.head()

,Year,AES,TEC,VDE,TES,GES
0,2014,88204418,14684731,1606386,68605877,7991074.0
1,2015,87413763,12041074,1234936,50259819,4964478.0
2,2016,80762094,12841033,1093190,49879543,7113989.0
3,2017,85314258,12208062,1177032,44945303,8531952.0
4,2018,84351328,12261018,1863726,47747092,10072767.0


In [91]:
df_sum.astype({'Year': 'int'}).dtypes

Year      int32
AES       int64
TEC       int64
VDE       int64
TES       int64
GES     float64
dtype: object

In [92]:
df_melted = df_sum.melt(id_vars=['Year'], var_name='Sources', value_name='MW')
df_melted.head()

,Year,Sources,MW
0,2014,AES,88204418.0
1,2015,AES,87413763.0
2,2016,AES,80762094.0
3,2017,AES,85314258.0
4,2018,AES,84351328.0


In [94]:
final_chart1 = alt.Chart(df_melted,title='Structure of electricity generation').mark_bar().encode(
    x = alt.X('Year:N'),
    y = alt.Y('MW:Q'),
    color = alt.Color('Sources:N'),
    tooltip = alt.Tooltip('MW')
).properties(width = 950, 
              height = 500, 
              background = '#FFFFFF', 
              padding = 25
             ).configure_view(strokeWidth = 0
             ).configure_axis(labelColor = "#999999",
                              gridWidth = 1,
                              titleFont = 'Helvetica',
                              labelFont = 'Helvetica',
                              titleFontSize = 14,
                              labelFontSize = 14
             ).configure_text(font = 'Helvetica'
             ).configure_title(
                                align='center',
                                anchor='start',
                                offset = 20,
                                fontSize = 18,
                                dx = 100,
                                fontWeight = 'bold'
             ).configure_legend(
                                titleFontSize=18,
                                labelFontSize=15
                              )

# final_chart.save("chart.html")
final_chart1

alt.Chart(...)

# 2. Як залежить споживання електроенергії від дня року та години доби?

Для зображення цих даних я вирішила зробити два графіки. Залежність споживання електроенергії від дня року показати на heatmap, адже будь-який інший спосіб дав би в результаті надто перевантажений даними чарт. На heatmap легко можна визначити день (точка по осі місяців і чисел) та за допомогою кольору дізнатися кількість спожитої електроенергії. Колір не є найкращим способом передачі даних, проте тут знову ж таки можна скористатися tooltip. Для зображення залежності споживання електроенергії від години доби я використала звичайний bar chart, цей графік є простим для читання та сприйняття і добре підходить під це завдання.

In [95]:
df2 = df.copy(deep=False)
df2.drop(['Year', 'AES','TEC', 'VDE', 'TES', 'GES'], axis = 1, inplace = True)
df2.head()

,Time/Date,CONSUMPTION
0,24-31.12.2020,16693
1,23-31.12.2020,17805
2,22-31.12.2020,18870
3,21-31.12.2020,19887
4,20-31.12.2020,20387


In [96]:
df2['hour'] = df['Time/Date'].apply(lambda x: str(x)[:2])
df2['hour'] = df2['hour'].apply(lambda x:  int(x[0]) if str(x)[-1] == "-" else int(x))
df2['Day_of_year'] = df['Time/Date'].apply(lambda x: str(x.split("-", 1)[1])[:5])
df2.drop(['Time/Date'], axis = 1, inplace = True)
df2.head()

,CONSUMPTION,hour,Day_of_year
0,16693,24,31.12
1,17805,23,31.12
2,18870,22,31.12
3,19887,21,31.12
4,20387,20,31.12


In [97]:
df_hours = df2.copy(deep=False)
df_hours.drop(['Day_of_year'], axis = 1, inplace = True)

df_hours = df_hours.groupby("hour").mean().round().reset_index()
df_hours.head()

,hour,CONSUMPTION
0,1,15193.0
1,2,14684.0
2,3,14466.0
3,4,14409.0
4,5,14558.0


In [98]:
df_hours.dtypes

hour             int64
CONSUMPTION    float64
dtype: object

In [99]:
part_chart1 = alt.Chart(df_hours,title='Average consuption of electricity per hour').mark_bar().encode(
    x = alt.X('hour:O'),
    y = alt.Y('CONSUMPTION:Q'),
    tooltip = alt.Tooltip('CONSUMPTION')
)

In [100]:
df_days = df2.copy(deep=False)
df_days.drop(['hour'], axis = 1, inplace = True)

df_days.head()

,CONSUMPTION,Day_of_year
0,16693,31.12
1,17805,31.12
2,18870,31.12
3,19887,31.12
4,20387,31.12


In [101]:
df_days = df_days.groupby("Day_of_year").mean().round().reset_index()
df_days.head()

,Day_of_year,CONSUMPTION
0,01.01,17750.0
1,01.02,20259.0
2,01.03,18930.0
3,01.04,17172.0
4,01.05,14260.0


In [102]:
df_days['Day_of_year'] = df_days['Day_of_year'].apply(lambda x: str(x.split(".", 1)[1])+"."+str(x.split(".", 1)[0]))
df_days.head()

,Day_of_year,CONSUMPTION
0,01.01,17750.0
1,02.01,20259.0
2,03.01,18930.0
3,04.01,17172.0
4,05.01,14260.0


In [103]:
part_chart2 = alt.Chart(df_days,title='Average consuption of electricity per day of the year').mark_rect().encode(
    alt.X('date(Day_of_year):O', title='day'),
    alt.Y('month(Day_of_year):O', title='month'),
    color='CONSUMPTION:Q',
    tooltip=['CONSUMPTION']
)

In [104]:
(part_chart1 | part_chart2).properties(
              padding = 25
             ).configure_view(strokeWidth = 0
             ).configure_axis(labelColor = "#999999",
                              gridWidth = 1,
                              titleFont = 'Helvetica',
                              labelFont = 'Helvetica',
                              titleFontSize = 14,
                              labelFontSize = 14
             ).configure_text(font = 'Helvetica'
             ).configure_title(
                                align='center',
                                anchor='start',
                                offset = 20,
                                fontSize = 18,
                                dx = 100,
                                fontWeight = 'bold'
             ).configure_legend(
                                titleFontSize=18,
                                labelFontSize=15
                              )

alt.HConcatChart(...)

# 3. Як змінюється генерація електроенергії з різних джерел впродовж доби?

Для цього завдання я обрала line graph. Тут я також розглядала grouped bar chart, але на ньому було б надто багато  стовпців, і це б сильно ускладнило б порівняння і сприйняття інформації. Line graph виглядає досить простим та ненавантаженим, його легко читати та порівнювати джерела генерації електроенергії.

In [105]:
df3 = df.copy(deep=False)
df3['hour'] = df3['Time/Date'].apply(lambda x: str(x)[:2])
df3['hour'] = df3['hour'].apply(lambda x:  int(x[0]) if str(x)[-1] == "-" else int(x))
df3.drop(['Year', 'Time/Date','CONSUMPTION'], axis = 1, inplace = True)
df3.head()

,AES,TEC,VDE,TES,GES,hour
0,9235,2039,621,4942,385.0,24
1,9221,2159,707,5549,470.0,23
2,9249,2377,709,5906,1000.0,22
3,9256,2499,702,6329,909.0,21
4,9213,2521,702,6640,823.0,20


In [106]:
df3 = df3.groupby("hour").mean().round().reset_index()
df3.head()

,hour,AES,TEC,VDE,TES,GES
0,1,9534.0,1429.0,164.0,4883.0,414.0
1,2,9531.0,1425.0,165.0,4773.0,326.0
2,3,9530.0,1425.0,165.0,4722.0,315.0
3,4,9532.0,1424.0,164.0,4685.0,290.0
4,5,9535.0,1426.0,165.0,4698.0,323.0


In [107]:
df3_melted = df3.melt(id_vars=['hour'], var_name='Sources', value_name='MW')
df3_melted.head()

,hour,Sources,MW
0,1,AES,9534.0
1,2,AES,9531.0
2,3,AES,9530.0
3,4,AES,9532.0
4,5,AES,9535.0


In [108]:
chart3 = alt.Chart(df3_melted,title='Average electricity generation per hour').mark_line(point=True).encode(
    x = alt.X('hour:N'),
    y = alt.Y('MW:Q'),
    color = alt.Color('Sources:N'),
    tooltip=['MW']
).properties(width = 750, 
              height = 400,
              padding = 25
             ).configure_view(strokeWidth = 0
             ).configure_axis(labelColor = "#999999",
                              gridWidth = 1,
                              titleFont = 'Helvetica',
                              labelFont = 'Helvetica',
                              titleFontSize = 14,
                              labelFontSize = 14
             ).configure_text(font = 'Helvetica'
             ).configure_title(
                                align='center',
                                anchor='start',
                                offset = 20,
                                fontSize = 18,
                                dx = 100,
                                fontWeight = 'bold'
             ).configure_legend(
                                titleFontSize=18,
                                labelFontSize=15
                              )
chart3

alt.Chart(...)

# 4. Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

Це завдання я також розділила на два графіки. Для зображення споживання електроенергії впродовж доби у розрізі місяців року я використала heatmap, бо лише так я змогла показати години кожного місяця, зберігши при цьому читабельність. У цьому графіку недоліком є передача інформації кольором, проте можна скористатися tooltip. Я ще також розглядала layered stripplot, але в такому графіку було б складно порівнювати використання енергії у різних місяцях. Для зображення споживання електроенергії впродовж доби у розрізі пір року я вирішила намалювати line graph. У цьому графіку дуже легко порівняти використання енергії у певну годину різних пір року і також певної пори року у різні години. 


In [109]:
df4 = df.copy(deep=False)
df4['hour'] = df4['Time/Date'].apply(lambda x: str(x)[:2])
df4['hour'] = df4['hour'].apply(lambda x:  int(x[0]) if str(x)[-1] == "-" else int(x))
df4['month'] = df4['Time/Date'].apply(lambda x: str(x.split(".", 2)[1]))
df4['season'] = df4['month'].apply(lambda x: "winter" if int(x)in [12, 1, 2] else ("spring" if int(x)in [3, 4, 5] else ("summer" if int(x)in [6, 7, 8] else "autumn")))
df4.drop(['Year', 'Time/Date','AES','TEC','VDE','TES','GES'], axis = 1, inplace = True)
df4.head()

,CONSUMPTION,hour,month,season
0,16693,24,12,winter
1,17805,23,12,winter
2,18870,22,12,winter
3,19887,21,12,winter
4,20387,20,12,winter


In [110]:
df4_month = df4.copy(deep=False)
df4_month.drop(['season'], axis = 1, inplace = True)
df4_month.head()

,CONSUMPTION,hour,month
0,16693,24,12
1,17805,23,12
2,18870,22,12
3,19887,21,12
4,20387,20,12


In [111]:
df4_month = df4_month.groupby(["month", "hour"]).mean().round().reset_index()
df4_month.head()

,month,hour,CONSUMPTION
0,01,1,17855.0
1,01,2,17307.0
2,01,3,17048.0
3,01,4,16985.0
4,01,5,17183.0


In [112]:
heatmap_chart = alt.Chart(df4_month,title='Average electricity consumption per months and hours').mark_rect().encode(
    alt.X('hour:N', title='hour'),
    alt.Y('month:N', title='month'),
    color='CONSUMPTION:Q',
    tooltip=['CONSUMPTION']
)


In [113]:
df4_season = df4.copy(deep=False)
df4_season.drop(['month'], axis = 1, inplace = True)
df4_season.head()

,CONSUMPTION,hour,season
0,16693,24,winter
1,17805,23,winter
2,18870,22,winter
3,19887,21,winter
4,20387,20,winter


In [114]:
df4_season = df4_season.groupby(["season", "hour"]).mean().round().reset_index()
df4_season.head()

,season,hour,CONSUMPTION
0,autumn,1,14731.0
1,autumn,2,14262.0
2,autumn,3,14063.0
3,autumn,4,14028.0
4,autumn,5,14238.0


In [115]:
season_chart4 = alt.Chart(df4_season,title='Average electricity consumption per seasons and per hours').mark_line(point=True).encode(
    x = alt.X('hour:N'),
    y = alt.Y('CONSUMPTION:Q', 
              scale = alt.Scale(
                  zero = False
              )),
    color = alt.Color('season:N'),
    tooltip=['CONSUMPTION', 'season', 'hour']
)

In [116]:
(heatmap_chart | season_chart4).properties(
              padding = 25
             ).configure_view(strokeWidth = 0
             ).configure_axis(labelColor = "#999999",
                              gridWidth = 1,
                              titleFont = 'Helvetica',
                              labelFont = 'Helvetica',
                              titleFontSize = 14,
                              labelFontSize = 14
             ).configure_text(font = 'Helvetica'
             ).configure_title(
                                align='center',
                                anchor='start',
                                offset = 20,
                                fontSize = 18,
                                dx = 100,
                                fontWeight = 'bold'
             ).configure_legend(
                                titleFontSize=18,
                                labelFontSize=15
                              )

alt.HConcatChart(...)

# 5. Як змінюється споживання електроенергії впродовж тижня?

Для цього завдання я скористалася bar chart. Цей графік легко читається, на ньому легко порівняти дні тижня між собою та визначити конкретну кількість спожитої електроенергії.

In [117]:
df5 = df.copy(deep=False)
df5.drop(['Year','AES','TEC','VDE','TES','GES'], axis = 1, inplace = True)
df5['Time/Date'] = df5['Time/Date'].apply(lambda x: str(x.split("-", 1)[1]))
df5.head()

,Time/Date,CONSUMPTION
0,31.12.2020,16693
1,31.12.2020,17805
2,31.12.2020,18870
3,31.12.2020,19887
4,31.12.2020,20387


In [118]:
df5['weekday'] = df5['Time/Date'].apply(lambda x: datetime.datetime.strptime(x, "%d.%m.%Y").weekday())
df5.drop(['Time/Date'], axis = 1, inplace = True)
df5 = df5.groupby(["weekday"]).mean().round().reset_index()
weekday_names = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
df5['weekday'] = df5['weekday'].apply(lambda x: weekday_names[x])

df5.head(7)

,weekday,CONSUMPTION
0,Monday,17168.0
1,Tuesday,17311.0
2,Wednesday,17385.0
3,Thursday,17396.0
4,Friday,17307.0
5,Saturday,16743.0
6,Sunday,16193.0


In [119]:
chart5 = alt.Chart(df5,title='Average electricity consumption per weekdays').mark_bar().encode(
    x = alt.X('weekday:N'),
    y = alt.Y('CONSUMPTION:Q'),
    tooltip = alt.Tooltip('CONSUMPTION')
).properties(width = 950, 
              height = 500, 
              background = '#FFFFFF', 
              padding = 25
             ).configure_view(strokeWidth = 0
             ).configure_axis(labelColor = "#999999",
                              gridWidth = 1,
                              titleFont = 'Helvetica',
                              labelFont = 'Helvetica',
                              titleFontSize = 14,
                              labelFontSize = 14
             ).configure_text(font = 'Helvetica'
             ).configure_title(
                                align='center',
                                anchor='start',
                                offset = 20,
                                fontSize = 18,
                                dx = 100,
                                fontWeight = 'bold'
             ).configure_legend(
                                titleFontSize=18,
                                labelFontSize=15
                              )

chart5

alt.Chart(...)